In [1]:
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import datetime
import shutil
from pathlib import Path
import trackpy
from iris.time import PartialDateTime
import cartopy.crs as ccrs
import xarray as xr
import netCDF4 as nc
import scipy
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure

#import packages that output memory usage:
#from sys import getsizeof

import tobac #tobac package cloned from https://github.com/tobac-project/tobac.git

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import datasets
mask = xr.open_dataset('/data/users/hgilmour/tracking/code/tobac_sensitivity/Save/mask_2005_01.nc')
mask = mask.segmentation_mask
mask #segmentation mask from tracking on Jan 2005 with [240, 200] and [1975, 10]

precip = xr.open_dataset('/data/users/hgilmour/total_precip/precip_1h/precip_2005_01.nc')
precip = precip.stratiform_rainfall_flux #precip dataset for Jan 2005 (NEED TO LATER CONVERT FROM KG M-2 S-1 TO MM/HR (X3600))

tracks = pd.read_hdf('/data/users/hgilmour/tracking/code/tobac_sensitivity/Save/tracks_2005_01.h5', 'table')

tb = xr.open_dataset('/data/users/hgilmour/tb/2005/tb_2005_01.nc')
tb = tb.toa_outgoing_longwave_flux

vert_vel = xr.open_dataset('/data/users/hgilmour/omega/2005/jan_2005_new_combine/omega_merge_01_2005_NEWMERGE1.nc')
vert_vel = vert_vel.dz_dt
#vert_vel = vert_vel.upward_air_velocity
#vert_vel = vert_vel[:,1:,:] # making sure there are 1360 latitudes rather than 1361


In [3]:
vert_vel = vert_vel[:-1,:,1:,:]
vert_vel

<xarray.DataArray 'dz_dt' (t: 240, p: 1, latitude: 1360, longitude: 1360)>
[443904000 values with dtype=float32]
Coordinates:
  * t          (t) datetime64[ns] 2005-01-01 ... 2005-01-30T21:00:00
  * longitude  (longitude) float32 274.9 275.0 275.0 275.1 ... 329.9 329.9 330.0
  * latitude   (latitude) float32 -40.02 -39.98 -39.94 ... 14.94 14.98 15.02
  * p          (p) float32 500.0
Attributes:
    standard_name:  upward_air_velocity
    long_name:      W COMPNT OF WIND ON P LEV/UV GRID
    units:          m s-1
    grid_mapping:   rotated_pole
    source:         Unified Model Output (Vn10.6):
    name:           dz_dt
    title:          W COMPNT OF WIND ON P LEV/UV GRID
    date:           01/01/02
    time:           00:00
    north_pole:     [180.  90.]

In [4]:
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,forecast_reference_time,forecast_period,cell,time_cell
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,-64.032043,295.967957,295.967957,1,0 days 00:00:00
1,0,26,319.360864,876.726404,7359,240,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-27.105936,-49.532534,310.467466,310.467466,2,0 days 00:00:00
2,0,57,465.549812,1014.466367,5384,240,3,2005-01-01 00:30:00,2005-01-01 00:30:00,-21.185283,-43.954061,316.045939,316.045939,-1,NaT
3,0,73,763.319602,705.760312,47759,240,4,2005-01-01 00:30:00,2005-01-01 00:30:00,-9.125606,-56.456667,303.543333,303.543333,-1,NaT
4,0,86,577.470368,717.240701,2132,240,5,2005-01-01 00:30:00,2005-01-01 00:30:00,-16.652501,-55.991687,304.008313,304.008313,-1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9076,743,241,882.032521,698.342702,7910,240,9077,2005-01-31 23:30:00,2005-01-31 23:30:00,-4.317733,-56.757057,303.242943,303.242943,-1,NaT
9077,743,252,940.225439,833.293263,3228,240,9078,2005-01-31 23:30:00,2005-01-31 23:30:00,-1.960920,-51.291560,308.708440,308.708440,-1,NaT
9078,743,272,1046.778633,1275.959222,20583,240,9079,2005-01-31 23:30:00,2005-01-31 23:30:00,2.354484,-33.363590,326.636410,326.636410,-1,NaT
9079,743,320,1097.697734,916.296729,3373,240,9080,2005-01-31 23:30:00,2005-01-31 23:30:00,4.416709,-47.929939,312.070061,312.070061,-1,NaT


In [5]:
vert_vel[1,:,:,:].t

<xarray.DataArray 't' ()>
array('2005-01-01T03:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    t        datetime64[ns] 2005-01-01T03:00:00
Attributes:
    standard_name:  time
    long_name:      t
    axis:           T

In [6]:
mask.shape == precip.shape # checking whether the mask and precip files have the same shape
# next steps won't work if not


True

In [7]:
# Copy tracks dataset into new tracks_precip dataset to append precip data to
tracks = tracks.copy()

In [8]:
# Add columns to the tracks_precip dataframe ready to append data to later
tracks['total_precip'] = 0 #total precip from any precipitating pixel
tracks['rain_flag'] = 0 # total number of pixels that meet the 1mm/hr threshold
tracks['light_precip'] = 0 # total rain from all pixels where the rainfall threshold of 1 mm/hr is met
tracks['heavy_precip'] = 0 # total rain from all pixels where the heavy rainfall threshold of 10 mm/hr is met
tracks['extreme_precip'] = 0 # total rain from all pixels where the extreme rainfall threshold of 50 mm/hr is met
tracks['max_precip'] = 0 # maximum rainfall rate found over the masked area at that timstep
tracks['mean_precip_total'] = 0 # mean rainfall rate found over whole masked area (including non rainy pixels)

tracks['mean_precip'] = 0 # mean rainfall rate found over pixels that meet the precipitation threshold (> 1 mm/hr)


# Add cold core filter columns
tracks['tb_min'] = 0
tracks['tb_mean'] = 0
tracks['cold_core_flag'] = 0
tracks['tb_210'] = 0
tracks['tb_200'] = 0
tracks['tb_190'] = 0

# Add columns for vertical velocity (w)
tracks['w_max'] = 0 # maximum vertical velocity
tracks['w_mean'] = 0 # mean vertical velocity
tracks['w_mean_up'] = 0 # mean vertical velocity of only positive values (i.e. upward motion)

# # Add columns for colocated pixels of cold core <= 200K and precip > 1mm/hr
# tracks['colocated_pixels_lightrain'] = 0
# tracks['colocated_CC_proportion_lightrain'] = 0
# tracks['colocated_precip_proportion_lightrain'] = 0

# # Add columns for colocated pixels of cold core <= 200K and heavy_precip > 10mm/hr
# tracks['colocated_pixels_heavyrain'] = 0
# tracks['colocated_CC_proportion_heavyrain'] = 0
# tracks['colocated_precip_proportion_heavyrain'] = 0

# # Add columns for colocated pixels of cold core <= 200K and extreme precip > 50mm/hr
# tracks['colocated_pixels_extremerain'] = 0
# tracks['colocated_CC_proportion_extremerain'] = 0
# tracks['colocated_precip_proportion_extremerain'] = 0


In [9]:
# remove cell values with -1 from tracks dataset
tracks = tracks[tracks.cell >= 0]
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,...,mean_precip,tb_min,tb_mean,cold_core_flag,tb_210,tb_200,tb_190,w_max,w_mean,w_mean_up
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,...,0,0,0,0,0,0,0,0,0,0
1,0,26,319.360864,876.726404,7359,240,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-27.105936,...,0,0,0,0,0,0,0,0,0,0
7,0,274,994.600931,652.721131,3251,240,8,2005-01-01 00:30:00,2005-01-01 00:30:00,0.241288,...,0,0,0,0,0,0,0,0,0,0
11,0,326,1136.057059,1330.881404,3448,240,12,2005-01-01 00:30:00,2005-01-01 00:30:00,5.970260,...,0,0,0,0,0,0,0,0,0,0
12,1,7,179.948634,523.925062,4138,240,13,2005-01-01 01:30:00,2005-01-01 01:30:00,-32.752130,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9062,742,351,1126.995562,1076.654673,15042,240,9063,2005-01-31 22:30:00,2005-01-31 22:30:00,5.603271,...,0,0,0,0,0,0,0,0,0,0
9064,743,28,506.670650,519.857317,40563,240,9065,2005-01-31 23:30:00,2005-01-31 23:30:00,-19.519889,...,0,0,0,0,0,0,0,0,0,0
9070,743,97,627.143818,585.244022,13066,240,9071,2005-01-31 23:30:00,2005-01-31 23:30:00,-14.640726,...,0,0,0,0,0,0,0,0,0,0
9074,743,208,801.606743,355.515817,3839,240,9075,2005-01-31 23:30:00,2005-01-31 23:30:00,-7.574979,...,0,0,0,0,0,0,0,0,0,0


In [10]:
tracks[tracks.cell == 8]

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,...,mean_precip,tb_min,tb_mean,cold_core_flag,tb_210,tb_200,tb_190,w_max,w_mean,w_mean_up
7,0,274,994.600931,652.721131,3251,240,8,2005-01-01 00:30:00,2005-01-01 00:30:00,0.241288,...,0,0,0,0,0,0,0,0,0,0
23,1,246,997.302790,649.673525,3985,240,24,2005-01-01 01:30:00,2005-01-01 01:30:00,0.350713,...,0,0,0,0,0,0,0,0,0,0
40,2,207,1000.085394,647.171876,4462,240,41,2005-01-01 02:30:00,2005-01-01 02:30:00,0.463408,...,0,0,0,0,0,0,0,0,0,0
53,3,151,1003.611792,643.427105,4439,240,54,2005-01-01 03:30:00,2005-01-01 03:30:00,0.606227,...,0,0,0,0,0,0,0,0,0,0
68,4,153,1001.760375,637.916268,3516,240,69,2005-01-01 04:30:00,2005-01-01 04:30:00,0.531246,...,0,0,0,0,0,0,0,0,0,0
82,5,161,1002.426140,633.101328,1978,240,83,2005-01-01 05:30:00,2005-01-01 05:30:00,0.558210,...,0,0,0,0,0,0,0,0,0,0
97,6,178,999.329845,589.566796,2196,240,98,2005-01-01 06:30:00,2005-01-01 06:30:00,0.432808,...,0,0,0,0,0,0,0,0,0,0
110,7,166,1005.332832,573.307600,2083,240,111,2005-01-01 07:30:00,2005-01-01 07:30:00,0.675929,...,0,0,0,0,0,0,0,0,0,0


In [11]:
cells = np.unique(tracks.cell.values)
cells

array([   1,    2,    8,   12,   14,   27,   28,   45,   48,   50,   54,
         56,   57,   59,   60,   62,   64,   66,   70,   73,   89,   91,
         98,  104,  106,  116,  118,  122,  124,  127,  128,  130,  134,
        135,  137,  138,  151,  154,  162,  165,  178,  185,  195,  204,
        207,  212,  215,  222,  223,  224,  226,  234,  247,  271,  276,
        287,  293,  296,  306,  307,  309,  324,  326,  339,  345,  354,
        358,  364,  369,  371,  376,  396,  399,  406,  407,  408,  409,
        413,  414,  418,  429,  434,  437,  441,  442,  443,  445,  449,
        451,  453,  458,  459,  467,  492,  498,  500,  514,  516,  518,
        522,  524,  529,  539,  541,  544,  545,  548,  549,  551,  552,
        554,  558,  566,  567,  577,  584,  587,  588,  590,  596,  601,
        603,  605,  608,  611,  615,  619,  621,  622,  624,  630,  631,
        634,  636,  638,  646,  647,  652,  666,  670,  673,  677,  681,
        682,  689,  695,  698,  716,  721,  725,  7

In [16]:
## TESTING LOOP FOR A SINGLE UNIQUE CELL (E.G. NUMBER 12) ##

## rainfall thresholds to use within filtering loop ##
precip_threshold = 1 #mm/hr
heavy_precip_threshold = 10 # mm/hr
extreme_precip_threshold = 50 # mm/hr (based on Marengo, J. A., Ambrizzi, T., Alves, L. M., Barreto, N. J., Simões Reboita, M., & Ramos, A. M. (2020). Changing trends in rainfall extremes in the metropolitan area of São Paulo: causes and impacts. Frontiers in Climate, 2, 3.)
precip_area = 25 # threshold for the minimum number of grid points that must be precipitating for a track to remain (and not be dropped from the tracks dataset)

cold_threshold = 200

## other parameters that need to be defined before loop ##
s = generate_binary_structure(2,2) # need this in loop later on
removed = 0 # need this for loop later on


cell = 566
subset = tracks[tracks.cell == cell]
precipitation_flag = 0
cold_core_flag = 0
#w_frame = 0 # assigning the frame number of the vertical velocity dataset to be 0 (i.e. vert_vel[w_frame,:,:,:] = vert_vel[0,:,:,:])

for feature in subset.feature.values: #find all the feature values for that unique cell / track (the feature value is a unique value for each feature in a frame /timestep)
    print("Feature:", feature)
    for frame in subset.frame[subset.feature == feature]: #find the frame / timestep that corresponds to the feature number
        print("Frame:", frame)
        print("Frame timestamp: ", str(subset['timestr'][subset.frame == frame]))
        if mask.shape == precip.shape:
        
            seg = mask[frame,:,:] #printing the segmentation mask which occurs in the same frame as the feature value
            #print(seg)
            prec = precip[frame,:,:] #printing the precip timesteps which occurs in the same frame as the feature value
            #print(prec)
            brightness_temp = tb[frame,:,:] # printing the tb timesteps which occurs in the same frame as the feature value
            brightness_temp = tb[frame,:,:] # printing the tb timesteps which occurs in the same frame as the feature value


            featureid = subset.feature[subset.frame == frame].values[0] #find the feature number at each timestep / frame of the cells lifetime (it changes over time and doesn't stay constant)
            #print('featureid: {}'.format(featureid)) #we now know all the feature numbers that belong to a single cell over its lifetime
            
            
            labels, num_labels = ndimage.label(seg, structure = s) #this line uses ndimage package for image processing. It generates arrays of numbers and decides what are joined together and what aren't.
            # In other words, it does image segmentation tasks, such as finding connected components and labeling objects in an image.
            # (i.e. it generates the locations of all contiguous fields of the segmentation mask that belong to a specific cell at a specific timestep and gives it a label. The number of labels is also recorded (the number of segmented areas in the timestep))

            if featureid not in seg: #check that the feature id number at each timestep is within the segmentation mask, if not, it is ignored and we continue
                continue
            else:

                label = np.unique(labels[seg == featureid])[0] #put a label where the labels match for both the feature id and the segmentation mask
                seg_mask = seg.where(labels == label)

                #create coordinates from mask
                seg_mask.coords['mask'] = (('longitude', 'latitude'), seg_mask.data)

                #apply mask to precip dataset
                precip_values = prec.where(seg_mask.coords['mask'].values > 0) # creating a new dataset called 'precip_values' with only the precip values where the seg_mask pixel is labelled as greater than 0 (i.e. the MCS region)
                #print('precip values: {}'.format(precip_values)
                array = precip_values.values.flatten() * 3600 # precip values are converted to 1D numpy array and multiplied by 3600 to convert from kg m-2 s-1 to mm / hr
                values = array[~np.isnan(array)] #removes NaNs from the precip array for further calculations
                #print(values)

                total_precip = np.nansum(values[values > 0]) #working out the total precip associated with the mask. First, values of 0 are removed to only consider precipitating pixels. Then np.nansum is used to compute the sum of all precipitating values within the mask.
                #print('total precip: {}'.format(total_precip))
                subset['total_precip'][(subset.feature == featureid) & (subset.frame == frame)  & (subset.cell == cell)] = total_precip

                rain_features = values[values >= precip_threshold].shape[0] #number of pixels within the mask that meet the 1 mm/hr precip threshold
                #print('rain features: {}'.format(rain_features))
                subset['rain_flag'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = rain_features

                subset['light_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = np.nansum(values[values >= precip_threshold]) #total rain from all pixel where the rainfall threshold of 1 mm/hr is met

                subset['heavy_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = np.nansum(values[values >= heavy_precip_threshold]) #total rain from all pixel where the heavy rainfall threshold of 10 mm/hr is met
                rain_features_heavy = values[values >= heavy_precip_threshold].shape[0] #number of pixels within the mask that meet the heavy rainfall threshold of 10 mm/hr

                subset['extreme_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = np.nansum(values[values >= extreme_precip_threshold]) #total rain from all pixel where the extreme rainfall threshold of 50 mm/hr is met
                rain_features_extreme = values[values >= extreme_precip_threshold].shape[0] #number of pixels within the mask that meet the extreme rainfall threshold of 50 mm/hr

                subset['max_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values.max()

                subset['mean_precip_total'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values.mean()

                subset['mean_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values[values > precip_threshold].mean()



                # cold core filter using the same image processing mask ##
                values_tb = brightness_temp.where(seg_mask.coords['mask'].values > 0)
                array_tb = values_tb.values.flatten() # precip values are converted to 1D numpy array and multiplied by 3600 to convert from kg m-2 s-1 to mm / hr
                values_tb = array_tb[~np.isnan(array_tb)] #removes NaNs from the precip array for further calculations
                print(values_tb[values_tb <= 190].shape[0])
                #array = values_tb.to_dataframe
                #values_tb = array().toa_outgoing_longwave_flux #Tb values in 1D array format to use in section below:

                subset['tb_mean'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values_tb.mean()

                subset['tb_min'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values_tb.min()

                subset['tb_210'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = (values_tb[values_tb <= 210]).shape[0]
                
                subset['tb_200'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = (values_tb[values_tb <= 200]).shape[0]             

                subset['tb_190'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = (values_tb[values_tb <= 190]).shape[0]  


                # ADDING IN IF STATEMENT FOR VERTICAL VELOCITY EVERY 3 TIMESTEPS#

                subset['datetime'] = pd.to_datetime(subset['timestr'][subset.frame == frame]) # the time at the hourly frame

                # vert_vel_df = pd.DataFrame()

                # vert_vel_df['datetime'] = vert_vel[:,:,:,:].t

                # for time in vert_vel_df['datetime'].dt.strftime('%Y-%m-%d %H:30:00'): # adding in the 30 so that its the hour of the vert_vel dataset + 30 mins to match up with the 1 hourly data
    
                #     if subset['datetime'][subset.frame == frame].to_string(index=False)==time:
                #         #print(time)

                #         for i in vert_vel_df.index:
                #             if (vert_vel[i,:,:,:].t.dt.strftime('%Y-%m-%d %H:30:00')) == time:
                #                 print("w_frame:" ,i)
                #                 w_frame = i

                #                 vel_w = vert_vel[w_frame,:,:,:] #find the vertical velocity values for the frame in the vert_vel dataset that corresponds to the frame in the hourly datasets (i.e if the original frame is at 03:00, then find the w values in the vert_vel dataset that are also at 03:00. THEY WILL BE DIFFERENT FRAME NUMBERS BECAUSE THE ORIGINAL IS 1-HOURLY AND W IS 3-HOURLY!!!)

                #                 #hour = int(subset.datetime.dt.hour[subset.frame == frame]) # this finds the hour of the curret timestep (01:30 = 1, 02:30 = 2, 03:30 = 3 etc...)
                #                 #time_list = [0,3,6,9,12,15,18,21] # specify the list of hours in the vertical velocity 3 hourly dataset


                #                 #if hour in time_list: # if the hour is available in the vertical velocity time list...
                #                     #vel_w = vert_vel[w_frame,:,:,:] #find the vertical velocity values for the frame in the vert_vel dataset that corresponds to the frame in the hourly datasets (i.e if the original frame is at 03:00, then find the w values in the vert_vel dataset that are also at 03:00. THEY WILL BE DIFFERENT FRAME NUMBERS BECAUSE THE ORIGINAL IS 1-HOURLY AND W IS 3-HOURLY!!!)

                #                 print("original frame timestamp: ", str(subset['timestr'][subset.frame == frame]))
                #                 print("w frame number: ", w_frame)
                #                 print("w frame timestamp: ", str(vert_vel[w_frame,:,:,:].t))

                #                 values_vel = vel_w.where(seg_mask.coords['mask'].values > 0) # extract only the w values within the segmented cell
                #                 array_vel = values_vel.values.flatten()
                #                 values_vel = array_vel[~np.isnan(array_vel)] # convert these into a 1D array with no nans

                #                 subset['w_max'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values_vel.max() #maximum vertical velocity over the segmented area
                #                 subset['w_mean'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values_vel.mean() # mean vertical velocity over the segmented area

                #                 w_up = values_vel[values_vel >= 0.000000001] # isolating just positive values of w (just upward motion and not downward)

                #                 subset['w_mean_up'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = w_up.mean() # mean upward vertical velocity over the segmented area

                    # ADD ANOTHER TIMESTEP ONTO W_FRAME FOR 3HRLY VERTICAL VELOCITY 
                    #w_frame = w_frame + 1  #add 1 to the vertical_velocity frame ready for the next time the hours are the same

                    # else:
                    #     print("Skipping this frame and assinging NaN") # if the hour doesn't correspond with one of the 3-hourly values, then assign NaNs to the dataframe
                    #     subset['w_max'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = float('NaN')
                    #     subset['w_mean'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = float('NaN')
                    #     subset['w_mean_up'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = float('NaN')
                        
                        

                # Assigning cold core flag
                if values_tb.min() <= cold_threshold:
                    cold_core_flag += 1
                    subset['cold_core_flag'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = cold_core_flag

                if rain_features >= precip_area: # if the number of precipitating pixels exceeds the miniumum pixel number... 
                    precipitation_flag += rain_features # add rain pixels to the precipitation flag


if cold_core_flag == 0:
    subset = subset.drop(subset[subset.cell == cell].index)
    removed += 1 #print the number of tracks that have been removed from the original dataset

    #save subset to deleted tracks folder
    #subset.to_hdf('Save/deleted_tracks/cold_core/tracks_2005_01_cell_{}'.format(cell), 'table')
    
if precipitation_flag == 0: #if the minumum precipitating pixel thresholds aren't met...
    #remove corresponding cell from the tracks dataframe
    subset = subset.drop(subset[subset.cell == cell].index)
    removed += 1 #print the number of tracks that have been removed from the original dataset

    #save subset to deleted tracks folder
    #subset.to_hdf('Save/deleted_tracks/precip/tracks_2005_01_cell_{}'.format(cell), 'table')

else:
    print("Completed cell ", cell)    
    # save precip track files
    #subset.to_hdf('Save/CC&PF/tracks_2005_01_cell_{}.h5'.format(cell), 'table')

Feature: 2441
Frame: 187
Frame timestamp:  2440    2005-01-08 19:30:00
Name: timestr, dtype: object
76
Feature: 2458
Frame: 188
Frame timestamp:  2457    2005-01-08 20:30:00
Name: timestr, dtype: object
43
Feature: 2476
Frame: 189
Frame timestamp:  2475    2005-01-08 21:30:00
Name: timestr, dtype: object
Feature: 2495
Frame: 190
Frame timestamp:  2494    2005-01-08 22:30:00
Name: timestr, dtype: object
57
Feature: 2511
Frame: 191
Frame timestamp:  2510    2005-01-08 23:30:00
Name: timestr, dtype: object
28
Feature: 2525
Frame: 192
Frame timestamp:  2524    2005-01-09 00:30:00
Name: timestr, dtype: object
18
Feature: 2537
Frame: 193
Frame timestamp:  2536    2005-01-09 01:30:00
Name: timestr, dtype: object
0
Feature: 2553
Frame: 194
Frame timestamp:  2552    2005-01-09 02:30:00
Name: timestr, dtype: object
0
Feature: 2569
Frame: 195
Frame timestamp:  2568    2005-01-09 03:30:00
Name: timestr, dtype: object
0
Feature: 2580
Frame: 196
Frame timestamp:  2579    2005-01-09 04:30:00
Name: ti

In [17]:
subset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,...,tb_min,tb_mean,cold_core_flag,tb_210,tb_200,tb_190,w_max,w_mean,w_mean_up,datetime
2440,187,384,1020.988148,440.259308,2486,240,2441,2005-01-08 19:30:00,2005-01-08 19:30:00,1.309972,...,187.007629,206.558380,1,1722,937,76,0,0,0,NaT
2457,188,314,1010.565974,445.137264,4660,240,2458,2005-01-08 20:30:00,2005-01-08 20:30:00,0.887871,...,187.207977,209.432831,2,2752,1106,43,0,0,0,NaT
2475,189,320,992.480319,419.092208,10519,240,2476,2005-01-08 21:30:00,2005-01-08 21:30:00,0.155401,...,0.000000,0.000000,0,0,0,0,0,0,0,NaT
2494,190,274,983.300866,384.144758,17265,240,2495,2005-01-08 22:30:00,2005-01-08 22:30:00,-0.216365,...,187.623856,216.060287,3,5806,1488,57,0,0,0,NaT
2510,191,242,979.939520,383.080082,18282,240,2511,2005-01-08 23:30:00,2005-01-08 23:30:00,-0.352501,...,189.341019,217.271072,4,5380,1862,28,0,0,0,NaT
2524,192,209,975.908811,381.135807,17746,240,2525,2005-01-09 00:30:00,2005-01-09 00:30:00,-0.515744,...,188.449448,218.327621,5,5331,1392,18,0,0,0,NaT
2536,193,196,970.977125,375.715296,16479,240,2537,2005-01-09 01:30:00,2005-01-09 01:30:00,-0.715476,...,191.451996,219.957947,6,3186,773,0,0,0,0,NaT
2552,194,171,971.214561,412.920750,11019,240,2553,2005-01-09 02:30:00,2005-01-09 02:30:00,-0.705860,...,192.301071,224.379303,7,896,38,0,0,0,0,NaT
2568,195,181,965.123420,408.428235,9353,240,2569,2005-01-09 03:30:00,2005-01-09 03:30:00,-0.952551,...,192.357651,225.448395,8,741,294,0,0,0,0,NaT
2579,196,167,957.832252,400.384179,8691,240,2580,2005-01-09 04:30:00,2005-01-09 04:30:00,-1.247845,...,192.607117,224.340561,9,1028,486,0,0,0,0,NaT


In [ ]:
print(subset['datetime'][subset.frame == frame])

timestamp = subset['datetime'][subset.frame == frame]

2635   2005-01-09 09:30:00
Name: datetime, dtype: datetime64[ns]


In [ ]:
vert_vel_df = pd.DataFrame()

vert_vel_df['datetime'] = vert_vel[:,:,:,:].t

In [ ]:
vert_vel[67,:,:,:].t

<xarray.DataArray 't' ()>
array('2005-01-09T09:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    t        datetime64[ns] 2005-01-09T09:00:00
Attributes:
    standard_name:  time
    long_name:      t
    axis:           T

In [ ]:
for i in vert_vel_df.index:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239


In [ ]:
for time in vert_vel_df['datetime'].dt.strftime('%Y-%m-%d %H:30:00'): # adding in the 30 so that its the hour of the vert_vel dataset + 30 mins to match up with the 1 hourly data
    
    if subset['datetime'][subset.frame == frame].to_string(index=False)==time:
        print(time)

        for i in vert_vel_df.index:
            if (vert_vel[i,:,:,:].t.dt.strftime('%Y-%m-%d %H:30:00')) == time:
                print("w_frame:" ,i)
                w_frame = i


2005-01-09 09:30:00
w_frame: 67


In [ ]:
print(vert_vel_df[vert_vel_df.datetime == time])

Empty DataFrame
Columns: [datetime]
Index: []


In [ ]:
vert_vel_df['datetime']

0     2005-01-01 00:00:00
1     2005-01-01 03:00:00
2     2005-01-01 06:00:00
3     2005-01-01 09:00:00
4     2005-01-01 12:00:00
              ...        
235   2005-01-30 09:00:00
236   2005-01-30 12:00:00
237   2005-01-30 15:00:00
238   2005-01-30 18:00:00
239   2005-01-30 21:00:00
Name: datetime, Length: 240, dtype: datetime64[ns]

In [ ]:
print(list(np.where(subset['datetime'][subset.frame == frame].reset_index(drop=True).equals(vert_vel_df['datetime'].reset_index(drop=True)))))

[array([], dtype=int64)]


In [ ]:
print(list(np.where(vert_vel_df["datetime"].equals(timestamp))))

[array([], dtype=int64)]


In [ ]:
if vert_vel_df['datetime'].equals(subset['datetime'][subset.frame == frame]):
    print(timestamp)

In [ ]:
for time in vert_vel_df['datetime'].dt.strftime('%Y-%m-%d %H:30:00'):
    print(time)
    for day in subset['datetime'][subset.frame == frame].dt.strftime('%m'):
        print(day)
        print(subset['datetime'][subset.frame == frame].dt.strftime('%d'))
        print(vert_vel_df['datetime'].dt.strftime('%d'))
        if subset['datetime'][subset.frame == frame].dt.strftime('%d').equals(vert_vel_df['datetime'].dt.strftime('%d')):
            print('THIS WORKS !!!!!!')
            print(vert_vel_df.index)
        else:
            continue

2005-01-01 00:30:00
01
2635    09
Name: datetime, dtype: object
0      01
1      01
2      01
3      01
4      01
       ..
235    30
236    30
237    30
238    30
239    30
Name: datetime, Length: 240, dtype: object
2005-01-01 03:30:00
01
2635    09
Name: datetime, dtype: object
0      01
1      01
2      01
3      01
4      01
       ..
235    30
236    30
237    30
238    30
239    30
Name: datetime, Length: 240, dtype: object
2005-01-01 06:30:00
01
2635    09
Name: datetime, dtype: object
0      01
1      01
2      01
3      01
4      01
       ..
235    30
236    30
237    30
238    30
239    30
Name: datetime, Length: 240, dtype: object
2005-01-01 09:30:00
01
2635    09
Name: datetime, dtype: object
0      01
1      01
2      01
3      01
4      01
       ..
235    30
236    30
237    30
238    30
239    30
Name: datetime, Length: 240, dtype: object
2005-01-01 12:30:00
01
2635    09
Name: datetime, dtype: object
0      01
1      01
2      01
3      01
4      01
       ..
235    3